## **CÁLCULO DE ÍNDICE DE SEVERIDAD DE INCENDIO**

<img src="https://github.com/romina-gonzalez-musso/Severidad_incendios/blob/main/_images/2_PyGEE.png?raw=true" width="400" height="90">

#### **MODO DE SELECCIÓN DE IMÁGENES:** por rango de fechas
Gonzalez Musso, Romina / Versión: Julio 2025

## **LIBRERIAS**

In [ ]:
## EN COLAB -----------------------
# Instalar cada vez que inicia Colab
pip install "pycrs"

# Importar librerías
import ee
import geemap

# Autenticar el usuario de GEE
ee.Authenticate()

# Inicializar GEE con el Cloud Project
ee.Initialize(project='mi-cloud-project')

In [1]:
## EN NOTEBOOK LOCAL -----------------------
# Importar librerías

# import ee
# import geemap
# ee.Initialize()

## **AREA DE ESTUDIO**

Importar un shape local que contenga el área de estudio donde se va a hacer la búsqueda de imágenes

In [ ]:
## EN COLAB -----------------------
# from google.colab import files
uploaded = files.upload()

roi = geemap.shp_to_ee("area.shp")

In [2]:
## EN NOTEBOOK LOCAL -----------------------
# roi = geemap.shp_to_ee("/home/romina/Descargas/2025-08_Taller_APN_Severidad_incendios/Inputs/area.shp")

## **FECHAS**
En este caso, se hará un compuesto de imágenes (*composite*) a partir de las imágenes presentes en un rango de fecha indicado por el usuario y un porcentaje mínimo de nubosidad.

In [3]:
# Fechas pre y post incendio
pre_fire = ['2024-11-15', '2024-12-01']
post_fire = ['2025-03-15', '2025-04-05']

# Porcentaje de nubosidad másximo
clouds_percent = 30

## **COLECCIÓN DE IMÁGENES**
Se usarán imágenes *Surface Reflectance* de [Sentintel 2](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR_HARMONIZED?hl=es-419)

In [4]:
# SENTINEL PRE-FIRE
s2_pre_Col = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')\
    .filterBounds(roi)\
    .filterDate(pre_fire[0], pre_fire[1])\
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', clouds_percent))\
    .sort('CLOUDY_PIXEL_PERCENTAGE')

# SENTINEL POST-FIRE
s2_post_Col = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')\
    .filterBounds(roi)\
    .filterDate(post_fire[0], post_fire[1])\
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', clouds_percent))\
    .sort('CLOUDY_PIXEL_PERCENTAGE')

# Ver número de imágenes
print("La colección de imágenes Pre-incendio con menos de" ,clouds_percent, "% de nubosidad tiene", s2_pre_Col.size().getInfo(), "imágenes")
print("La colección de imágenes Post-incendio con menos de" ,clouds_percent, "% de nubosidad tiene", s2_post_Col.size().getInfo(), "imágenes")

La colección de imágenes Pre-incendio con menos de 30 % de nubosidad tiene 9 imágenes
La colección de imágenes Post-incendio con menos de 30 % de nubosidad tiene 22 imágenes


In [5]:
# Crear el mosaico
s2_pre = s2_pre_Col.mosaic().clip(roi)
s2_post = s2_post_Col.mosaic().clip(roi)

# Ver fechas de las imágenes que componen el mosaico
def get_date(image):
    return ee.Feature(None, {
        'date': image.date().format('YYYY-MM-dd')})

dates_pre = s2_pre_Col.map(get_date)
dates_pre = dates_pre.aggregate_array('date').getInfo()
dates_pre = sorted(list(set(dates_pre)))

dates_post = s2_post_Col.map(get_date)
dates_post = dates_post.aggregate_array('date').getInfo()
dates_post = sorted(list(set(dates_post)))

print("Fechas de las imágenes del mosaico Pre-incendio:", dates_pre)
print("Fechas de las imágenes del mosaico Post-incendio:", dates_post)

Fechas de las imágenes del mosaico Pre-incendio: ['2024-11-18', '2024-11-30']
Fechas de las imágenes del mosaico Pre-incendio: ['2025-03-15', '2025-03-18', '2025-03-23', '2025-03-25', '2025-04-04']


#### **Visualizar compuestos**

In [6]:
# VISUALIZAR CON CORTINILLA PARA COMPARAR
Map = geemap.Map()

vis_true_color = {'bands': ['B4', 'B3', 'B2'], 'max': 2000, 'min': 10, 'gamma': 1.5}
vis_swir = {'bands': ['B12', 'B8A', 'B4'], 'max': 4000, 'min': 150, 'gamma': 1.5}

# Crear capas como tile layer
left_layer = geemap.ee_tile_layer(s2_pre.visualize(**vis_true_color), name='Pre-fire', shown=True)
right_layer = geemap.ee_tile_layer(s2_post.visualize(**vis_true_color), name='Post-fire', shown=True)

# Agregar la cortinilla interactiva
Map.split_map(left_layer=left_layer, right_layer=right_layer)

# Centrar el mapa en tu ROI
Map.centerObject(roi, zoom=10)
Map

Map(center=[-41.46897551910003, -71.7231425154939], controls=(ZoomControl(options=['position', 'zoom_in_text',…

## **Opcional: aplicar máscaras de agua y vegetación**

#### **Máscara de agua**

In [7]:
# Máscara de agua con Global Surface Water
gsw = ee.Image("JRC/GSW1_2/GlobalSurfaceWater")
water_mask = gsw.select('seasonality').lt(11).unmask(1).clip(roi)

#### **Máscara de vegetación**

In [8]:
def getIndexes(image):
    ndwi = image.normalizedDifference(['B3', 'B5']).rename('NDWI')
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    return image.addBands([ndvi, ndwi])

indices = getIndexes(s2_pre)

# Máscaras de vegetación con NDVI
veg_mask = indices.select('NDVI').gte(0.08).unmask(1).clip(roi)

In [9]:
# Aplicar las máscaras
s2_pre_mask = s2_pre.mask(water_mask).mask(veg_mask)
s2_post_mask = s2_post.mask(water_mask).mask(veg_mask)

In [10]:
# VISUALIZAR CON CORTINILLA PARA COMPARAR
Map = geemap.Map()

vis_true_color = {'bands': ['B4', 'B3', 'B2'], 'max': 2000, 'min': 10, 'gamma': 1.5}
vis_swir = {'bands': ['B12', 'B8A', 'B4'], 'max': 4000, 'min': 150, 'gamma': 1.5}

# Crear capas como tile layer
left_layer = geemap.ee_tile_layer(s2_post.visualize(**vis_true_color), name='Pre-fire', shown=True)
right_layer = geemap.ee_tile_layer(s2_post_mask.visualize(**vis_true_color), name='Post-fire', shown=True)

# Agregar la cortinilla interactiva
Map.split_map(left_layer=left_layer, right_layer=right_layer)

# Centrar el mapa en tu ROI
Map.centerObject(roi, zoom=10)
Map

Map(center=[-41.46897551910003, -71.7231425154939], controls=(ZoomControl(options=['position', 'zoom_in_text',…

## **Cálculo del Índice NBR (Índice Normalizado de Área Quemada)**

![imagen](https://github.com/romina-gonzalez-musso/Severidad_Incendio-Steffen-Martin22/blob/master/_images/1_NBR_formula.jpg?raw=true)

In [11]:
preNBR = s2_pre_mask.normalizedDifference(['B8A', 'B12'])
postNBR = s2_post_mask.normalizedDifference(['B8A', 'B12'])

# Delta NBR
dNBR_unscaled = preNBR.subtract(postNBR)

#### **Calcular el dNBR offset**

Link al paper: https://www.mdpi.com/2072-4292/6/3/1827

In [12]:
# dNBRoffset --> -0.0277 (o 27.7 si se escala por 1000)
dNBRoffset = 0.0277

# dNBR sin escalar
dNBR_unscaled = dNBR_unscaled.subtract(0.0277)

# Scale product to USGS standards
dNBR = dNBR_unscaled.multiply(1000)

#### **Visualizar imágenes y dNBR**

In [13]:
Map = geemap.Map(center=[-40,-71], zoom=4)

# DELTA NBR
Map.addLayer(dNBR, {'max': 1200, 'min': 0}, 'dNBR')

# NBR en GRIS
vis_grey = {'palette': ['white', 'black'], 'max': 1, 'min': -1}

Map.addLayer(preNBR, vis_grey, 'PreNBR')
Map.addLayer(postNBR, vis_grey, 'PostNBR')

# IMÁGENES
vis_true_color = {'bands': ['B4', 'B3', 'B2'], 'max': 2000, 'min': 10, 'gamma': 1.5}
vis_swir = {'bands': ['B12', 'B8A', 'B4'], 'max': 4000, 'min': 150, 'gamma': 1.5}

Map.addLayer(s2_pre, vis_swir, 'Pre')
Map.addLayer(s2_post, vis_swir, 'Post')

Map.centerObject(roi, 10)

Map

Map(center=[-41.46897551910003, -71.7231425154939], controls=(WidgetControl(options=['position', 'transparent_…

#### **ESCALA USGS**
Link: https://un-spider.org/advisory-support/recommended-practices/recommended-practice-burn-severity/in-detail/normalized-burn-ratio

<img src="https://github.com/romina-gonzalez-musso/Severidad_Incendio-Steffen-Martin22/blob/master/_images/1_classes.png?raw=true" width="800">

In [14]:
# Escala de colores USGS
sld_intervals = \
  '<RasterSymbolizer>' + \
    '<ColorMap type="intervals" extended="false" >' + \
      '<ColorMapEntry color="#ffffff" quantity="-500" label="-500"  />' + \
      '<ColorMapEntry color="#7a8737" quantity="-250" label="-250"  />' + \
      '<ColorMapEntry color="#acbe4d" quantity="-100" label="-100" />' + \
      '<ColorMapEntry color="#0ae042" quantity="100" label="100" />' + \
      '<ColorMapEntry color="#fff70b" quantity="270" label="270" />' + \
      '<ColorMapEntry color="#ffaf38" quantity="440" label="440" />' + \
      '<ColorMapEntry color="#ff641b" quantity="660" label="660" />' + \
      '<ColorMapEntry color="#a41fd6" quantity="2000" label="2000" />' + \
    '</ColorMap>' + \
  '</RasterSymbolizer>'

# Escala de colores USGS modificado por Romi
sld_intervals_romi = \
  '<RasterSymbolizer>' + \
    '<ColorMap type="intervals" extended="false" >' + \
      '<ColorMapEntry color="#000000" quantity="-500" label="-500"  />' + \
      '<ColorMapEntry color="#000000" quantity="-250" label="-250"  />' + \
      '<ColorMapEntry color="#000000" quantity="-100" label="-100" />' + \
      '<ColorMapEntry color="#000000" quantity="100" label="100" />' + \
      '<ColorMapEntry color="#fff70b" quantity="270" label="270" />' + \
      '<ColorMapEntry color="#ffaf38" quantity="440" label="440" />' + \
      '<ColorMapEntry color="#ff641b" quantity="660" label="660" />' + \
      '<ColorMapEntry color="#a41fd6" quantity="2000" label="2000" />' + \
    '</ColorMap>' + \
  '</RasterSymbolizer>'

In [15]:
Map = geemap.Map()

# Visualización para Sentinel-2
vis_true_color = {'bands': ['B4', 'B3', 'B2'], 'max': 2000, 'min': 10, 'gamma': 1.5}

# Crear capas
left_layer = geemap.ee_tile_layer(s2_post.visualize(**vis_true_color), name='Post-fire', shown=True)
right_layer = geemap.ee_tile_layer(dNBR.sldStyle(sld_intervals_romi), name='dNBR classified', shown=True)

# Cortinilla
Map.split_map(left_layer=left_layer, right_layer=right_layer)
Map.centerObject(roi, zoom=10)
Map

Map(center=[-41.46897551910003, -71.7231425154939], controls=(ZoomControl(options=['position', 'zoom_in_text',…

## **EXPORTAR ASSETS**

In [ ]:
## IMAGEN PRE-INCENDIO
s2_pre_bands = ee.Image(s2_pre).select(
    ['B2',  'B3',   'B4',   'B8A',   'B12'],
    ['blue', 'green', 'red', 'nir2', 'swir2']) # Blue (10m) - Green (10m) - Red (10m) - NIR2 - SWIR2 (20m)

# Armar el task
from ee.batch import Export
task = Export.image.toDrive(
            image= s2_pre_bands,
            description = "S2_pre_composite",
            folder= "GEE_export",
            scale= 10,
            crs = 'EPSG:4326',
            region = roi.geometry())

# Iniciar el task
task.start()

In [ ]:
## IMAGEN POST-INCENDIO
s2_post_bands = ee.Image(s2_post).select(
    ['B2',  'B3',   'B4',   'B8A',   'B12'],
    ['blue', 'green', 'red', 'nir2', 'swir2']) # Blue (10m) - Green (10m) - Red (10m) - NIR2 - SWIR2 (20m)

# Armar el task
from ee.batch import Export
task = Export.image.toDrive(
            image= s2_post_bands,
            description = "S2_post_composite",
            folder= "GEE_export",
            scale= 10,
            crs = 'EPSG:4326',
            region = roi.geometry())

# Iniciar el task
task.start()

In [ ]:
## dNBR
from ee.batch import Export
task = Export.image.toDrive(
            image= dNBR,
            description = "dNBR_composite",
            folder= "GEE_export",
            scale= 10,
            crs = 'EPSG:4326',
            region = roi.geometry())

# Iniciar el task
task.start()